# Explainable AI (XAI)

### Data Mining Project 2024/25

Authors: Nicola Emmolo, Simone Marzeddu, Jacopo Raffi

In [55]:
import pandas as pd
import numpy as np
import seaborn as sb
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

from sklearn.metrics import classification_report
import shap
import pickle

from interpret import set_visualize_provider
from interpret.provider import InlineProvider
from interpret import show
from tqdm import tqdm
import random

from interpret.blackbox import LimeTabular

from xailib.explainers.lore_explainer import LoreTabularExplainer

import dice_ml

In [56]:
RANDOM_STATE = 42
train_set = pd.read_csv('../data/ml_datasets/oversampling/train_set.csv').sample(frac = 1, random_state=RANDOM_STATE) # shuffling the data so not to introduce bias
val_set = pd.read_csv('../data/ml_datasets/oversampling/val_set.csv').sample(frac = 1, random_state=RANDOM_STATE) # shuffling the data so not to introduce bias
test_set = pd.read_csv('../data/ml_datasets/oversampling/test_set.csv')

dev_set = pd.concat([train_set, val_set]).reset_index(drop=True)

In [57]:
dev_set['race_season%autumn'] = dev_set['race_season%autumn'].astype(int)
dev_set['race_season%spring'] = dev_set['race_season%spring'].astype(int)
dev_set['race_season%summer'] = dev_set['race_season%summer'].astype(int)
dev_set['race_season%winter'] = dev_set['race_season%winter'].astype(int)

test_set['race_season%autumn'] = test_set['race_season%autumn'].astype(int)
test_set['race_season%spring'] = test_set['race_season%spring'].astype(int)
test_set['race_season%summer'] = test_set['race_season%summer'].astype(int)
test_set['race_season%winter'] = test_set['race_season%winter'].astype(int)

In [58]:
dev_label = dev_set.pop('label')
test_label = test_set.pop('label')

In [59]:
model = RandomForestClassifier(class_weight=None, criterion='entropy', max_features=8, min_samples_leaf=5, min_samples_split=20, n_estimators=150)
model.fit(dev_set, dev_label)

test_predicitions = model.predict(test_set)
dev_predictions = model.predict(dev_set)

In [60]:
model_report = classification_report(test_label, test_predicitions, output_dict=True)

## Lime

In [61]:
set_visualize_provider(InlineProvider())

In [62]:
# Reorder columns of dev_set accordingly
dev_set = dev_set[model.feature_names_in_]

explanation_algorithm = LimeTabular(model, dev_set)

In [63]:
indices = np.argwhere((dev_predictions == 1) & (dev_label == 0))
indices[:10]

array([[  10],
       [ 210],
       [ 233],
       [ 451],
       [ 682],
       [ 838],
       [1144],
       [1317],
       [1644],
       [2184]])

In [64]:
indices = np.argwhere((dev_predictions == 0) & (dev_label == 1))
indices[:10]

array([[ 803],
       [ 935],
       [2283],
       [2441],
       [3590],
       [3876],
       [4678],
       [5836],
       [6238],
       [6621]])

In [65]:
# Class 1 - Correct Prediciton

lime_result = explanation_algorithm.explain_local(dev_set[700:1000], dev_label[700:1000])
lime_result.visualize(2) #702

In [66]:
# Class 1 - Wrong Prediciton

lime_result = explanation_algorithm.explain_local(dev_set[700:1000], dev_label[700:1000])
lime_result.visualize(103) #803

In [67]:
# Class 0 - Correct Prediciton

lime_result = explanation_algorithm.explain_local(dev_set[700:1000], dev_label[700:1000])
lime_result.visualize(1) #701

In [68]:
# Class 0 - Wrong Prediciton

lime_result = explanation_algorithm.explain_local(dev_set[700:1000], dev_label[700:1000])
lime_result.visualize(138) #838

## Lore

In [69]:
explanation_algorithm = LoreTabularExplainer(model)
configuration = {
    "neigh_type": "rndgen", # metodo di generazione del dataset locale (in questo caso random)
    "size": 1000, # dimensione del dataset locale generato
    "ocr": 0.1, # probabilità di "cambiamento" di una feature durante la perturbazione
    "ngen": 100 # numero di generazioni 
}

lore_train_data = pd.concat((dev_set, pd.DataFrame(dev_predictions, columns=["label"])), axis="columns")
explanation_algorithm.fit(lore_train_data, "label", config=configuration)

In [70]:
# Il modello ha predetto 1 e questa è la predizione corretta

explanation_lore = explanation_algorithm.explain(dev_set.iloc[702].values)
explanation_lore.plotRules()
print(explanation_lore.expDict["fidelity"])

0.9973872847580536


In [71]:
# Il modello ha predetto 0 ma la label è 1

explanation_lore = explanation_algorithm.explain(dev_set.iloc[803].values)
explanation_lore.plotRules()
print(explanation_lore.expDict["fidelity"])

0.9756372509804523


In [72]:
# Il modello ha predetto 0 e questa è la predizione corretta

explanation_lore = explanation_algorithm.explain(dev_set.iloc[701].values)
explanation_lore.plotRules()
print(explanation_lore.expDict["fidelity"])

0.9773748045498741


In [73]:
# Il modello ha predetto 1 ma la label è 0

explanation_lore = explanation_algorithm.explain(dev_set.iloc[838].values)
explanation_lore.plotRules()
print(explanation_lore.expDict["fidelity"])

0.997389121779106


## Counterfactuals Explainations

In [74]:
# need to convert data to the Dice format
dice_train_dataset = dev_set.copy()
dice_train_dataset["label"] = dev_label
dice_dataset = dice_ml.Data(
    dataframe=dice_train_dataset,
    outcome_name="label",
    continuous_features=dice_train_dataset.columns.tolist()[:-1]
)

dice_model = dice_ml.Model(
    model=model,
    backend="sklearn",  # dice supports several frameworks, e.g., pytorch, tensorflow
)

# the Dice object interfaces the counterfactual search
dice_explanation_algorithm = dice_ml.Dice(
    dice_dataset,
    dice_model,
    method="random"  # random search for counterfactuals
)

In [75]:
explanation_dice = dice_explanation_algorithm.generate_counterfactuals(
    # 702 ->1V, 701 -> 0v, 838 -> 1x, 803 -> 0x
    dice_train_dataset.drop("label", axis="columns").iloc[[702, 701, 838, 803]],
    total_CFs=10,  # number of counterfactuals to generate
    features_to_vary = list(set(dice_train_dataset.drop("label", axis="columns").columns) - set(["race_season%summer", "race_season%winter", "race_season%spring", "race_season%autumn"])),  # more feature constraints
)

100%|██████████| 4/4 [01:20<00:00, 20.03s/it]


In [76]:
explanation_dice.visualize_as_dataframe(show_only_changes=True)

Query instance (original outcome : 1)


,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,num_participants,race_season%autumn,race_season%spring,race_season%summer,race_season%winter,label
0,155.0,22.99169,3,0.001258,0.003405,0.11985,0.651808,656.454529,0.000061,0.025801,184,0,1,0,0,1



Diverse Counterfactual set (new outcome: 0)


,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,num_participants,race_season%autumn,race_season%spring,race_season%summer,race_season%winter,label
0,-,-,-,0.19996306,-,0.36555409,-,-,6.055730975659768e-05,-,-,-,-,-,-,0.0
1,-,-,-,-,-,-,-,9221.7,6.055730975659768e-05,0.9,-,-,-,-,-,0.0
2,-,21.153122,-,-,-,-,0.7179402,-,6.055730975659768e-05,-,-,-,-,-,-,0.0
3,-,20.318663,-,-,-,-,-,3243.0,6.055730975659768e-05,-,-,-,-,-,-,0.0
4,-,-,-,0.51746824,-,-,-,4714.1,6.055730975659768e-05,-,-,-,-,-,-,0.0
5,-,-,-,-,0.11127287,0.44827567,-,-,6.055730975659768e-05,-,-,-,-,-,-,0.0
6,-,19.358197,-,-,-,0.87629722,-,-,6.055730975659768e-05,-,-,-,-,-,-,0.0
7,42.6,-,-,-,-,-,0.4165874,-,6.055730975659768e-05,-,-,-,-,-,-,0.0
8,-,21.486016,-,-,-,-,-,-,6.055730975659768e-05,-,-,-,-,-,-,0.0
9,-,-,-,-,0.50728878,-,-,-,6.055730975659768e-05,-,-,-,-,-,-,0.0


Query instance (original outcome : 0)


...

,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,num_participants,race_season%autumn,race_season%spring,race_season%summer,race_season%winter,label
0,-,-,-,-,-,-,0.8023775,-,-,-,-,-,-,-,-,0.0
1,-,-,-,-,-,0.65699278,-,10956.8,-,-,-,-,-,-,-,0.0
2,-,-,-,-,0.02215157,-,-,-,-,-,-,-,-,-,-,0.0
3,210.0,-,-,-,-,-,-,-,-,-,-,-,-,-,-,0.0
4,-,-,-,-,0.67769326,-,-,-,-,0.2,-,-,-,-,-,0.0
5,-,-,-,0.0143384,-,-,-,-,-,-,-,-,-,-,-,0.0
6,-,23.784306,-,-,-,-,-,4226.1,-,-,-,-,-,-,-,0.0
7,-,-,0.0,-,0.52691575,-,-,-,-,-,-,-,-,-,-,0.0
8,-,-,-,-,-,-,0.6020503,-,-,-,-,-,-,-,-,0.0
9,-,-,-,-,-,0.83796909,-,-,-,-,-,-,-,-,-,0.0


Query instance (original outcome : 0)


,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,num_participants,race_season%autumn,race_season%spring,race_season%summer,race_season%winter,label
0,160.5,19.568235,2,0.01167,0.033873,0.234451,0.290661,232.270264,0.001495,0.510944,143,0,0,1,0,0



Diverse Counterfactual set (new outcome: 1)


,length,cyclist_bmi,cyclist_age_group,climb_percentage,race_physical_effort,race_prestige,previous_mean_position,previous_mean_delta,previous_mean_cp,cyclist_previous_experience,num_participants,race_season%autumn,race_season%spring,race_season%summer,race_season%winter,label
0,-,26.73462,-,-,-,-,-,-,-,-,-,-,-,-,-,1.0
1,-,24.551001,-,-,-,-,-,-,-,0.2,-,-,-,-,-,1.0
2,-,-,-,-,-,-,0.2101175,-,-,-,42.0,-,-,-,-,1.0
3,-,26.516773,-,-,-,-,-,-,-,-,-,-,-,-,-,1.0
4,-,26.027072,1.0,-,-,-,-,-,-,-,-,-,-,-,-,1.0
5,-,-,-,-,-,-,0.29980569999997125,-,-,-,8.0,-,-,-,-,1.0
6,-,-,-,-,-,-,-,-,-,-,95.0,-,-,-,-,1.0
7,-,17.556969,-,-,-,-,-,-,-,-,12.0,-,-,-,-,1.0
8,-,27.449866,-,-,-,-,-,-,-,-,-,-,-,-,-,1.0
9,-,-,-,-,-,-,-,-,0.2678114818,-,101.0,-,-,-,-,1.0


## Validation

In [77]:
pd.DataFrame(model_report)

,0,1,accuracy,macro avg,weighted avg
precision,0.903343,0.412479,0.847167,0.657911,0.838156
recall,0.922466,0.355460,0.847167,0.638963,0.847167
f1-score,0.912804,0.381853,0.847167,0.647329,0.842294
support,27446.000000,4203.000000,0.847167,31649.000000,31649.000000


In [78]:
features = list(dev_set.columns)
features

['length',
 'cyclist_bmi',
 'cyclist_age_group',
 'climb_percentage',
 'race_physical_effort',
 'race_prestige',
 'previous_mean_position',
 'previous_mean_delta',
 'previous_mean_cp',
 'cyclist_previous_experience',
 'num_participants',
 'race_season%autumn',
 'race_season%spring',
 'race_season%summer',
 'race_season%winter']

In [79]:
random_noise = np.random.rand(test_set.shape[0],)
noise_magnitudes = [eps / 10 for eps in range(1, 10)]

# We corrupt the test set with different noise magnitudes
# Each corruption (different magnitude) is stored in a dict
corruptions_by_feature_and_noise = dict()
for feature in features:
    for noise_magnitude in noise_magnitudes:
        corruptions_by_feature_and_noise[(feature, noise_magnitude)] = test_set.copy()
        corruptions_by_feature_and_noise[(feature, noise_magnitude)][feature] = corruptions_by_feature_and_noise[(feature, noise_magnitude)][feature] + random_noise * noise_magnitude

In [80]:
validations_by_feature_and_noise = dict()
for (feature, noise), corruption in corruptions_by_feature_and_noise.items():
    validations_by_feature_and_noise[(feature, noise)] = classification_report(test_label, model.predict(corruption), output_dict=True)

In [81]:
metric = "f1-score"
corruption_validation = pd.DataFrame([
    (feature, noise_magnitude, validations_by_feature_and_noise[(feature, noise_magnitude)]["macro avg"][metric])
    for feature, noise_magnitude in validations_by_feature_and_noise
    ],
    columns=["feature", "noise_magnitude", "performance"]
)
corruption_validation["difference"] = corruption_validation["performance"] - model_report["macro avg"][metric]
corruption_validation

,feature,noise_magnitude,performance,difference
0,length,0.1,0.647251,-0.000077
1,length,0.2,0.647285,-0.000043
2,length,0.3,0.647251,-0.000077
3,length,0.4,0.647295,-0.000034
4,length,0.5,0.647406,0.000077
...,...,...,...,...
130,race_season%winter,0.5,0.647329,0.000000
131,race_season%winter,0.6,0.647714,0.000385
132,race_season%winter,0.7,0.648023,0.000694
133,race_season%winter,0.8,0.648074,0.000745


In [82]:
corruption_validation.groupby("feature").describe()[["performance", "difference"]]

performance                                \
                                  count      mean       std       min   
feature                                                                 
climb_percentage                    9.0  0.609098  0.000300  0.608838   
cyclist_age_group                   9.0  0.647103  0.000499  0.645815   
cyclist_bmi                         9.0  0.641606  0.002546  0.639369   
cyclist_previous_experience         9.0  0.631183  0.008357  0.621289   
length                              9.0  0.647383  0.000121  0.647251   
num_participants                    9.0  0.647420  0.000065  0.647379   
previous_mean_cp                    9.0  0.601808  0.003684  0.598812   
previous_mean_delta                 9.0  0.647801  0.000262  0.647475   
previous_mean_position              9.0  0.538916  0.041623  0.498241   
race_physical_effort                9.0  0.610431  0.009555  0.602855   
race_prestige                       9.0  0.619428  0.012900  0.606444   
race_season%autumn                  9.0  0.647361  0.000203  0.647066   
race_season%spring                  9.0  0.647251  0.000141  0.646946   
race_season%summer                  9.0  0.647064  0.000352  0.646373   
race_season%winter                  9.0  0.647609  0.000348  0.647329   

                                                                     \
                                  25%       50%       75%       max   
feature                                                               
climb_percentage             0.608888  0.608989  0.609114  0.609726   
cyclist_age_group            0.647262  0.647329  0.647329  0.647329   
cyclist_bmi                  0.639632  0.640662  0.643962  0.645715   
cyclist_previous_experience  0.624786  0.629371  0.637346  0.645254   
length                       0.647285  0.647406  0.647457  0.647611   
num_participants             0.647379  0.647379  0.647438  0.647566   
previous_mean_cp             0.599553  0.601109  0.602100  0.611032   
previous_mean_delta          0.647586  0.647818  0.648032  0.648169   
previous_mean_position       0.508367  0.527127  0.557104  0.622556   
race_physical_effort         0.603982  0.608120  0.611877  0.633020   
race_prestige                0.610087  0.615411  0.625289  0.643472   
race_season%autumn           0.647329  0.647329  0.647329  0.647750   
race_season%spring           0.647253  0.647329  0.647329  0.647338   
race_season%summer           0.646894  0.647329  0.647329  0.647329   
race_season%winter           0.647329  0.647329  0.648023  0.648074   

                            difference                                \
                                 count      mean       std       min   
feature                                                                
climb_percentage                   9.0 -0.038231  0.000300 -0.038491   
cyclist_age_group                  9.0 -0.000225  0.000499 -0.001513   
cyclist_bmi                        9.0 -0.005722  0.002546 -0.007960   
cyclist_previous_experience        9.0 -0.016146  0.008357 -0.026040   
length                             9.0  0.000054  0.000121 -0.000077   
num_participants                   9.0  0.000091  0.000065  0.000050   
previous_mean_cp                   9.0 -0.045520  0.003684 -0.048517   
previous_mean_delta                9.0  0.000472  0.000262  0.000146   
previous_mean_position             9.0 -0.108413  0.041623 -0.149088   
race_physical_effort               9.0 -0.036898  0.009555 -0.044474   
race_prestige                      9.0 -0.027900  0.012900 -0.040885   
race_season%autumn                 9.0  0.000032  0.000203 -0.000263   
race_season%spring                 9.0 -0.000077  0.000141 -0.000383   
race_season%summer                 9.0 -0.000265  0.000352 -0.000956   
race_season%winter                 9.0  0.000281  0.000348  0.000000   

                                                                     
                                  25%       50%       75%       max

In [83]:
corruption_validation.groupby("noise_magnitude").describe()[["performance", "difference"]]

performance                                                    \
                      count      mean       std       min       25%       50%   
noise_magnitude                                                                 
0.1                    15.0  0.639302  0.013641  0.609726  0.638246  0.647251   
0.2                    15.0  0.633855  0.021289  0.581341  0.625949  0.647285   
0.3                    15.0  0.631074  0.026045  0.557104  0.618583  0.647251   
0.4                    15.0  0.628771  0.029865  0.539808  0.613703  0.647295   
0.5                    15.0  0.627345  0.032732  0.527127  0.612124  0.647329   
0.6                    15.0  0.625819  0.035760  0.513998  0.611070  0.646894   
0.7                    15.0  0.625017  0.037259  0.508367  0.609508  0.646719   
0.8                    15.0  0.624127  0.038860  0.501702  0.608075  0.646944   
0.9                    15.0  0.623569  0.039606  0.498241  0.607779  0.645815   

                                    difference                                \
                      75%       max      count      mean       std       min   
noise_magnitude                                                                
0.1              0.647329  0.647475       15.0 -0.008027  0.013641 -0.037603   
0.2              0.647329  0.647586       15.0 -0.013474  0.021289 -0.065988   
0.3              0.647329  0.647552       15.0 -0.016254  0.026045 -0.090225   
0.4              0.647329  0.648169       15.0 -0.018558  0.029865 -0.107520   
0.5              0.647329  0.648101       15.0 -0.019984  0.032732 -0.120202   
0.6              0.647385  0.648032       15.0 -0.021510  0.035760 -0.133331   
0.7              0.647511  0.648023       15.0 -0.022312  0.037259 -0.138961   
0.8              0.647468  0.648074       15.0 -0.023202  0.038860 -0.145627   
0.9              0.647295  0.648030       15.0 -0.023760  0.039606 -0.149088   

                                                         
                      25%       50%       75%       max  
noise_magnitude                                          
0.1             -0.009082 -0.000077  0.000000  0.000146  
0.2             -0.021380 -0.000043  0.000000  0.000257  
0.3             -0.028746 -0.000077  0.000000  0.000223  
0.4             -0.033626 -0.000034  0.000000  0.000841  
0.5             -0.035204  0.000000  0.000000  0.000772  
0.6             -0.036259 -0.000434  0.000056  0.000704  
0.7             -0.037820 -0.000609  0.000183  0.000694  
0.8             -0.039253 -0.000385  0.000139  0.000745  
0.9             -0.039550 -0.001513 -0.000034  0.000702

In [84]:
removed_feature_results = []

for feature in features:  # Ordine di importanza delle feature
    # Filtra il dataset per rimuovere la feature corrente
    reduced_train_data = dev_set.drop(columns=[feature])
    reduced_test_data = test_set.drop(columns=[feature])

    # Riaddestra il modello da zero
    model.fit(reduced_train_data, dev_label)

    # Valuta il modello sul dataset di test ridotto
    predictions = model.predict(reduced_test_data)
    report = classification_report(test_label, predictions, output_dict=True)

    # Salva i risultati
    removed_feature_results.append({
        "feature_removed": feature,
        "performance": report["macro avg"]["f1-score"]  # Usa la metrica desiderata
    })

# Converte i risultati in un dataframe per analisi
removed_feature_results_df = pd.DataFrame(removed_feature_results)

In [85]:
removed_feature_results_df.sort_values(by="performance", ascending=False)

,feature_removed,performance
13,race_season%summer,0.649692
3,climb_percentage,0.649218
2,cyclist_age_group,0.648383
12,race_season%spring,0.647757
9,cyclist_previous_experience,0.647020
14,race_season%winter,0.645694
11,race_season%autumn,0.645152
5,race_prestige,0.644944
0,length,0.643717
8,previous_mean_cp,0.643001
